<a href="https://colab.research.google.com/github/MrOlivo/Brutalism-UI/blob/main/Export_to_Cashew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime

accounts = pd.read_csv("/content/drive/MyDrive/Ivy Wallet/cashew/originals/accounts.csv")
#print(accounts.columns)

categories = pd.read_csv("/content/drive/MyDrive/Ivy Wallet/cashew/originals/categories.csv")
#print(categories.columns)

transactions = pd.read_csv("/content/drive/MyDrive/Ivy Wallet/cashew/originals/transactions.csv")
#print(transactions.columns)

transactions.loc[transactions['type'] == 'EXPENSE', 'amount'] = transactions['amount'] * -1

#transactions

In [ ]:
joined_entries = pd.merge(transactions, accounts, how='left', left_on='accountId', right_on='id')
df_merged = pd.merge(joined_entries, categories, how='left', left_on='categoryId', right_on='id')
df_merged['note'] = np.nan

#df_merged

In [ ]:
def reemplazar_texto(valor_condicional, valor_actual):
  if valor_condicional == "TRANSFER":
    return 'Transferencia'
  else:
    return valor_actual

df_merged['name_y'] = df_merged.apply(lambda row: reemplazar_texto(row['type'], row['name_y']), axis=1)

#df_merged

In [ ]:
def get_account_name(valor_condicional, valor_actual):
    if valor_condicional == "e770a1da-7f16-4c07-bb5f-ea0ba9c33cd8":
        return "Efectivo"
    elif valor_condicional == "b3f669d1-4064-46e0-865d-1285e19c80a3":
        return "BanCoppel"
    elif valor_condicional == "145331c4-b141-4053-aac9-386eb540f202":
        return "Caja Popular"
    elif valor_condicional == "dfc9acd9-3052-4fdf-8825-b13633485a6d":
        return "Nu Credito"
    elif valor_condicional == "cb95e20a-e747-4949-8e73-73a59fc00f48":
        return "Nu Ahorro"
    elif valor_condicional == "8b050316-925c-4019-bb5f-d4807c81d6ea":
        return "Billú"
    elif valor_condicional == "11e2343c-51ca-440e-b85c-c2ca26288804":
        return "BBVA"
    else:
        return valor_actual

def isNaN(num):
    return num != num

def add_note(currentRow):
  if not isNaN(currentRow['toAccountId']):
    return "Saldo transferido {} → {}".format(
        get_account_name(currentRow["accountId"], currentRow["name_x"]),
        get_account_name(currentRow["toAccountId"], currentRow["name_x"]))
  else:
    return currentRow['toAccountId']

def add_transfer_note(currentRow, accountId, message):
  if currentRow['type'] == 'TRANSFER':
    return message.format(
        get_account_name(accountId, currentRow["name_x"]))
  else:
    return currentRow['title']

In [ ]:
df_duplicated = df_merged[df_merged["type"] == "TRANSFER"].copy()

# Add meesage «<Account> Transfer In»
df_merged["title"] = df_merged.apply(
    lambda row: add_transfer_note(row, row["toAccountId"], "{} Transfer In"), axis=1
)
# Add meesage «<Account> Transfer Out»
df_duplicated["title"] = df_duplicated.apply(
    lambda row: add_transfer_note(row, row["accountId"], "{} Transfer Out"), axis=1
)

# Change movement type
df_merged["type"] = df_merged[df_merged["type"] == "TRANSFER"] = "INCOME"
df_duplicated["type"] = df_duplicated[df_duplicated["type"] == "TRANSFER"] = "EXPENSE"

# Change EXPENSEs to negative
df_duplicated["amount"] = df_duplicated["amount"] * -1

# Replace «account» value with «toAccount» value
df_merged["name_x"] = df_merged.apply(
    lambda row: get_account_name(row["toAccountId"], row["name_x"]), axis=1
)

In [ ]:
# Join both dataframes
df_concatenated = pd.concat([df_merged, df_duplicated])

# Adds note «Saldo transferido Cuenta A → Cuenta B»
df_concatenated["note"] = df_concatenated.apply(add_note, axis=1)

# Get only the needed columns
df_final = df_concatenated[["dateTime", "amount", "name_y", "title", "note", "name_x"]]
df_final.columns = ["Date", "Amount", "Category", "Title", "Note", "Account"]

df_final.sort_values(by=["Date", "Amount"], ascending=False, inplace=True)
df_final.reset_index(drop=True, inplace=True)

df_final

In [ ]:
from datetime import datetime

nombre_archivo = f"data_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

with open(
    "/content/drive/MyDrive/Ivy Wallet/cashew/{}.csv".format(nombre_archivo),
    "w",
    encoding="utf-8-sig",
) as f:
    df_final.to_csv(f)
